In [10]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup

In [11]:
def parser(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')

    mot = soup.find_all('span', class_='mot')
    mot = [i.text.strip() for i in mot]
    c1 = [i for i in mot if len(i) < 6]
    c2 = [i.split() for i in mot if len(i) > 5]
    c2 = [i for j in c2 for i in j]
    c3 = c1 + c2

    mot2 = soup.find_all('span', class_='mot2')
    mot2 = [i.text.strip() for i in mot2]
    d1 = [i for i in mot2 if len(i) < 6]
    d2 = [i.split() for i in mot2 if len(i) > 5]
    d2 = [i for j in d2 for i in j]
    d3 = d1 + d2

    content = c3 + d3
    return content

In [12]:
address_list = []
for i in range(1, 16):
    address = 'https://www.bestwordlist.com/5letterwordspage' + str(i) + '.htm'
    address_list.append(address)

address_list[0] = 'https://www.bestwordlist.com/5letterwords.htm'
# print(address_list)

In [13]:
lst = [parser(link) for link in address_list]
lst = [i for j in lst for i in j]
# print(lst)
# print(len(lst))

In [14]:
# get scores for every letter
from collections import Counter
letter_lst = [i for j in lst for i in j]
# print(letter_lst)

d = dict(Counter(letter_lst))
# print(d)

In [15]:
# determine max score
letter_scores = list(d.values())
max_score = max(letter_scores) * 5
# print(letter_scores)
# print(max_score)

# get frequency score
def frequency_score(word, scores_dict, max_score):
    scores_array = []
    for i in word:
        scores_array.append(scores_dict.get(i))

    scores_array = sum(scores_array) / max_score
    return scores_array

# print(frequency_score('ABACK', d, max_score))

In [16]:
# get dissimilary score
def dissim_score(word):
    word_set = set(word)
    word_score = len(word_set) / len(word)
    return word_score

# print(dissim_score('ABACK'))

In [17]:
import math

# frequency score array
frequency_score_array = [frequency_score(i, d, max_score) for i in lst]

# dissimilary score array
dissim_score_array = [dissim_score(i) for i in lst]

# final score array
final_score_array = list(np.add(frequency_score_array, dissim_score_array))
final_score_array_max = max(final_score_array)
final_score_array_min = min(final_score_array)
final_score_array = [(((i - final_score_array_min) / (final_score_array_max - final_score_array_min)) * 100) for i in final_score_array]
# print(final_score_array)

# create list of tuples
def merge(list1, list2):
    merged_list = list(zip(list1, list2))
    return merged_list

tup_lst = merge(lst, final_score_array)
tup_lst = [{"word": i.upper(), "score": round(j*10)} for i, j in tup_lst]
tup_lst = sorted(tup_lst, key=lambda d: d['score'])
# print(tup_lst)

In [18]:
# with open('data.json', 'w') as json_file:
#     json.dump(tup_lst, json_file)